# Projet maison n° 3

In [97]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

In [98]:
#on telecharge un jeu de données au format zip depuis une url et on extrait tout vers le dossier spécifié
import requests
from zipfile import ZipFile

def get_and_unzip_file_from_url(url,zip_name,directory_name):
    req = requests.get(url, verify=False)
    with open(zip_name,'wb') as output_file:
        output_file.write(req.content)
    with ZipFile(zip_name, 'r') as zipObj:
        zipObj.extractall(directory_name)

In [99]:
get_and_unzip_file_from_url('https://www.ssa.gov/oact/babynames/names.zip','US_baby_names.zip','US_baby_names')

/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ssa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [100]:
import os
import re

def df_names_us():
    df = pd.DataFrame()
    for file in os.listdir("US_baby_names"):
        if file.endswith(".txt"):
            year = re.findall("[0-9]+", file)#year est un array de string
            df_by_year = pd.read_csv("US_baby_names/"+file, 
                                     sep=',', 
                                     names=["name", "gender", "births"]
                                    )
            df_by_year["year"] = int(year[0])
            df = pd.concat([df, df_by_year], ignore_index=True)

    return df.sort_values(by=['year','gender','births','name'],ascending=[True,True,False,True],ignore_index=True).reindex(columns=["year", "name", "gender", "births"])# trie les ligne par année et réordonne les colonnes

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 

In [101]:
get_and_unzip_file_from_url('https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip','Prenom_Francais.zip','Prenom_Francais')

/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.insee.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [102]:
# names fr
def df_names_fr():
    df = pd.DataFrame()
    for file in os.listdir("Prenom_Francais"):
        if file.endswith(".csv"):
            df = ( pd.read_csv("Prenom_Francais/"+file, 
                            sep=';',
                            #names=["gender", "name", "year", "births"],
                            #header=0,
                            na_values=["XXXX","_PRENOMS_RARES"],
                            keep_default_na=False #pour eviter de supprimer le prénon Na
                           )
                  #Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
                  .rename(columns={"sexe": "gender", "preusuel": "name", "annais": "year", "nombre": "births"})
                  .reindex(columns=["year", "name", "gender", "births"])
                  
                  #Les lignes avec des données inutilisables doivent être supprimées
                  .dropna()
                  
                  #Mêmes valeurs pour la colonne "gender"
                  .assign(gender=lambda x: x.gender.map({2: 'F', 1: 'M'}))
                  
                  #Seuls les prénoms de 2 caractères et plus sont conservés
                  .query("name.str.len() >= 2", engine="python") 
                  
                  #La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
                  .assign(name=lambda x: x.name.str.title())# on n'utilise pas .capitalize() car on veut que chaque nom d'un nom composé ait une capitale
                  
                  #Mêmes dtypes pour les colonnes
                  .astype({"gender":"str", "name":"str", "year":"uint16", "births":"uint16"})
                  
                  #Mêmes dtypes pour les colonnes L'index du DataFrame doit aller de 0 à N-1
                  .sort_values(by=['year','gender','births','name'], ascending=[True,True,False,True], ignore_index=True)
                 )
            
    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [103]:
# taux de change
def df_taux_change(devises):
    df = ( pd.read_csv("Webstat_Export.csv",
                       sep=';',
                       skiprows=[0,1,3,4,5],#supprimes les lignes qui ne nous interessent pas
                                            #on ne garde que le ligne 2 car c'est la seule à contenir tous les ISO
                       na_values=["-"],
                       index_col=0,
                       decimal=',',
                       parse_dates=True
                      )
          #renomme les colones en fonction du code ISO
          .rename(columns=lambda x: re.findall("[A-Z]{3}", x)[0])
          #ordone par date des index ordre croissant
          .sort_index()
          #remplace les valeur na
          .fillna(0)
         )
    #df = pd.DataFrame([], columns=devises)
    return df.loc[:,devises]

### Tests

In [104]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [105]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [106]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 6.455s

OK
